In [1]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [2]:
PARTICIPANT = "TEST_VIDEO"
KEYPOINTS = ["right_wrist", "right_index"]

In [3]:
# Load keypoint data
keypoints_df = pd.read_csv('../Keypoints/' + PARTICIPANT + '_keypoints.csv', encoding='utf-8')
column_names = [
    "time_ms", 
    "right_shoulder_x", "right_shoulder_y", 
    "left_shoulder_x", "left_shoulder_y", 
    "right_elbow_x", "right_elbow_y", 
    "left_elbow_x", "left_elbow_y", 
    "right_wrist_x", "right_wrist_y", 
    "left_wrist_x", "left_wrist_y", 
    "right_eye_x", "right_eye_y", 
    "left_eye_x", "left_eye_y",
    "nose_x", "nose_y",
    "right_index_x", "right_index_y"
]

keypoints_df.columns = column_names

keypoints_df

,time_ms,right_shoulder_x,right_shoulder_y,left_shoulder_x,left_shoulder_y,right_elbow_x,right_elbow_y,left_elbow_x,left_elbow_y,right_wrist_x,...,left_wrist_x,left_wrist_y,right_eye_x,right_eye_y,left_eye_x,left_eye_y,nose_x,nose_y,right_index_x,right_index_y
0,33.0,0.201754,0.335460,0.309713,0.316838,0.218326,0.520750,0.321477,0.460743,0.304107,...,0.336435,0.567709,0.262751,0.204579,0.281319,0.206071,0.330839,0.601892,0.275076,0.225096
1,67.0,0.201921,0.335186,0.309394,0.316891,0.217597,0.520174,0.321468,0.462534,0.303681,...,0.336211,0.565990,0.267274,0.204831,0.283631,0.206566,0.330216,0.601913,0.280492,0.225105
2,100.0,0.202088,0.334913,0.309222,0.316889,0.216593,0.519490,0.321405,0.463053,0.302665,...,0.335854,0.563910,0.270195,0.205345,0.285242,0.207197,0.329703,0.602722,0.283303,0.225317
3,133.0,0.202186,0.334658,0.309030,0.316855,0.215805,0.517955,0.321377,0.463035,0.299517,...,0.335424,0.560803,0.271506,0.206027,0.286165,0.207978,0.326935,0.603452,0.284498,0.225847
4,167.0,0.202186,0.334740,0.308908,0.316859,0.215845,0.517773,0.321383,0.463097,0.299748,...,0.335282,0.558852,0.271768,0.206634,0.286852,0.208811,0.327096,0.604405,0.284808,0.226606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2527,84328.0,0.195131,0.323393,0.299869,0.319983,0.218879,0.499788,0.313037,0.478042,0.304334,...,0.323758,0.594267,0.272648,0.194354,0.284182,0.194909,0.327530,0.624214,0.287398,0.214793
2528,84362.0,0.195353,0.323813,0.300000,0.319982,0.218959,0.500091,0.313182,0.478122,0.304907,...,0.323841,0.595206,0.271735,0.194227,0.283889,0.194849,0.328207,0.624215,0.286600,0.214714
2529,84395.0,0.195695,0.323806,0.300172,0.320077,0.219256,0.500233,0.313226,0.478119,0.305145,...,0.323893,0.595257,0.271101,0.193322,0.283779,0.193897,0.328561,0.624247,0.285968,0.213618
2530,84429.0,0.195952,0.323918,0.300374,0.320098,0.219561,0.500700,0.313229,0.477947,0.305546,...,0.324148,0.595220,0.271436,0.193118,0.284271,0.193729,0.329119,0.624263,0.286479,0.213264


In [4]:
# Initialize an empty DataFrame to store final results
final_df = pd.DataFrame()

# Generate the new time series at 10 ms intervals
start_time = np.ceil(keypoints_df['time_ms'].iloc[0] / 10) * 10
end_time = np.floor(keypoints_df['time_ms'].iloc[-1] / 10) * 10
new_time_series = np.arange(start_time, end_time, 10)

# Loop through each keypoint specified in the KEYPOINTS list
for keypoint in KEYPOINTS:
    # Extracting original x and y coordinates
    x_original = keypoints_df[f'{keypoint}_x'].values
    y_original = keypoints_df[f'{keypoint}_y'].values
    time_original = keypoints_df['time_ms'].values

    # Interpolation for upsampling to 10 ms intervals
    interp_x = interp1d(time_original, x_original, kind='linear')
    interp_y = interp1d(time_original, y_original, kind='linear')
    x_upsampled = interp_x(new_time_series)
    y_upsampled = interp_y(new_time_series)

    # Smoothing using Savitzky-Golay filter
    window_length = 11  # Choose an appropriate window length
    polyorder = 3  # Choose an appropriate polynomial order
    x_smooth = savgol_filter(x_upsampled, window_length, polyorder)
    y_smooth = savgol_filter(y_upsampled, window_length, polyorder)

    # Speed calculation
    delta_time = np.diff(new_time_series)
    delta_x = np.diff(x_smooth) / delta_time
    delta_y = np.diff(y_smooth) / delta_time
    speed = np.sqrt(np.nan_to_num(delta_x ** 2 + delta_y ** 2) / delta_time)
    
    # Normalize speed to make it comparable across participants
    speed = speed / np.nanmax(speed)
    
    # if speed is below the 20th percentile, set it to 0
    speed[speed < np.nanpercentile(speed, 50)] = 0
    
    # smooth the speed
    speed = savgol_filter(speed, 11, 3)

    # Add to the final DataFrame
    temp_df = pd.DataFrame({
        'time_ms': new_time_series[:-1],
        f'{keypoint}_x': x_smooth[:-1],
        f'{keypoint}_y': y_smooth[:-1],
        f'{keypoint}_speed': speed,
        f'{keypoint}_delta_x': delta_x,
        f'{keypoint}_delta_y': delta_y, 
    })
    if final_df.empty:
        final_df = temp_df
    else:
        final_df = pd.merge(final_df, temp_df, on='time_ms', how='right')

final_df.head()

,time_ms,right_wrist_x,right_wrist_y,right_wrist_speed,right_wrist_delta_x,right_wrist_delta_y,right_index_x,right_index_y,right_index_speed,right_index_delta_x,right_index_delta_y
0,40.0,0.303906,0.590180,0.006968,0.000006,-0.000023,0.276141,0.225104,0.317009,0.000175,-1.062103e-06
1,50.0,0.303966,0.589952,0.007484,-0.000008,-0.000018,0.277888,0.225093,0.276687,0.000149,7.892571e-07
2,60.0,0.303882,0.589771,0.010898,-0.000022,-0.000014,0.279377,0.225101,0.236226,0.000125,2.789179e-06
3,70.0,0.303665,0.589632,0.016101,-0.000034,-0.000011,0.280630,0.225129,0.196309,0.000104,4.937662e-06
4,80.0,0.303326,0.589526,0.021989,-0.000045,-0.000008,0.281667,0.225178,0.157619,0.000084,7.234707e-06


In [5]:
print("Length of x_upsampled:", len(x_upsampled))
print("Length of y_upsampled:", len(y_upsampled))
print(len(new_time_series))

Length of x_upsampled: 8442
Length of y_upsampled: 8442
8442


In [6]:
# Save the aligned data as a CSV file
final_df.to_csv(f'../Upsampled/'+ PARTICIPANT + '_processed_data.csv', index=False)
final_df.head()

print(f'File saved as {PARTICIPANT}_processed_data.csv')

File saved as TEST_VIDEO_processed_data.csv


In [7]:
import plotly.graph_objects as go

# Create traces
trace0 = go.Scatter(x=final_df['time_ms'], y=final_df['right_wrist_speed'], mode='lines', name='Right Wrist Speed')
trace1 = go.Scatter(x=final_df['time_ms'], y=final_df['right_wrist_delta_x'], mode='lines', name='Right Wrist Raw Velocity X', yaxis='y2')
trace2 = go.Scatter(x=final_df['time_ms'], y=final_df['right_wrist_delta_y'], mode='lines', name='Right Wrist Raw Velocity Y', yaxis='y2')

# Create layout
layout = go.Layout(title='Right Wrist Speed and Velocity', xaxis=dict(title='Time (ms)'), yaxis1=dict(title='Speed', overlaying='y', side='left'))

# Create figure
fig = go.Figure(data=[trace0], layout=layout)

# Show figure
fig.show()